In [1]:
# pip install -U autogen-agentchat autogen-ext[openai,web-surfer]
# playwright install
#%pip install python.dotenv

## Example using the ROUND ROBING GROUPCHAT team with a USER PROXY AGENT and a MULTIMODAL WEBSURFER

In [1]:
import sys
import asyncio
from autogen_agentchat.agents import UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
from autogen_agentchat.teams import MagenticOneGroupChat
from dotenv import load_dotenv

from settings import llm_config
load_dotenv()


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


True

In [2]:
model_client = AzureOpenAIChatCompletionClient.load_component(llm_config)

## This requires USER to provide feedback as to what to do next

In [4]:
async def main() -> None:
    # The web surfer will open a Chromium browser window to perform web browsing tasks.
    web_surfer = MultimodalWebSurfer("web_surfer", model_client, headless=False, animate_actions=True)
    # The user proxy agent is used to get user input after each step of the web surfer.
    # NOTE: you can skip input by pressing Enter.
    user_proxy = UserProxyAgent("user_proxy")
    # The termination condition is set to end the conversation when the user types 'exit'.
    termination = TextMentionTermination("exit", sources=["user_proxy"])
    # Web surfer and user proxy take turns in a round-robin fashion.
    team = RoundRobinGroupChat([web_surfer, user_proxy], termination_condition=termination)
    try:
        # Start the team and wait for it to terminate.
        await Console(team.run_stream(task="find information about the latest iPhone"))
    finally:
        await web_surfer.close()
        await model_client.close()

await main()

---------- TextMessage (user) ----------
find information about the latest iPhone


/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:610: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- MultiModalMessage (web_surfer) ----------
I typed 'latest iPhone' into '0 characters out of 2000'.

The web browser is open to the page [latest iPhone - Search](https://www.bing.com/search?q=latest+iPhone&form=QBLH&sp=-1&lq=0&pq=latest+iphon&sc=12-12&qs=n&sk=&cvid=28CE1C1256A74E7E861503C71ED5ED19).
The viewport shows 17% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

Skip to content
latest iPhoneFrançais3
You could win a Million USD!
Claim Your Free Entry to Join the $2 Million Dollar Sweepstakes!
Claim free entry
Later
All
Search
Copilot
Shopping
Images
Videos
Maps
More
Tools
About 1,690,000 resultsSponsored
See Latest iPhoneApple iPhone 15, 256GB, Black (Renewed)$959.99Amazon CAFree shippingiPhone 16 Pro Max 256GB With 2-Yr Phone Plan On Bell Smartpay + Device Return Option$1,799.28 now$42/mo x 24
Bell MobilityiPhone 16 Pro 128GB Desert Titanium Unlocked- Apple$1,449.00Apple StoreFree shippingiPhone 15 128GB Pink Un

## This DOES NOT require user feedback as to what to do next, the orchestrator does that

In [3]:
async def main() -> None:

    surfer = MultimodalWebSurfer(
        "MultimodalWebSurfer",
        model_client=model_client,
        downloads_folder="./downs",
        debug_dir="./debug",
        headless = False,
        to_resize_viewport=True,
        description="A web surfing assistant that can browse and interact with web pages.",
        start_page="https://www.google.com",  # Optional: Initial page
        animate_actions=True,
        browser_data_dir="./browser_data",
    )
    
    team = MagenticOneGroupChat([surfer], model_client=model_client, max_turns=3)
    # Define a team
    # team = RoundRobinGroupChat([surfer], max_turns=3)
    #await Console(team.run_stream(task="Navigate to the AutoGen readme on GitHub."))
    #await Console(team.run_stream(task="Summarize latest updates from Accenture newsroowm."))
    #await Console(team.run_stream(task="Summarize latest news from venture beat all things in AI."))
    #await Console(team.run_stream(task="Find information about the latest iPhone."))
    await Console(team.run_stream(task="what NHL games are happening today, and tell me about the teams playing"))
    #await Console(team.run_stream(task="what is the weather like in Toronto today, and what is the weather like in New York today?"))
    await surfer.close()


if __name__ == "__main__":
    if sys.platform == "win32":
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
await main()

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:164: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:177: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- TextMessage (user) ----------
what NHL games are happening today, and tell me about the teams playing
---------- TextMessage (MagenticOneOrchestrator) ----------

We are working to address the following user request:

what NHL games are happening today, and tell me about the teams playing


To answer this request we have assembled the following team:

MultimodalWebSurfer: A web surfing assistant that can browse and interact with web pages.


Here is an initial fact sheet to consider:

### 1. GIVEN OR VERIFIED FACTS
- The request mentions NHL games happening "today."
- The request asks for information about the teams playing in these games.

### 2. FACTS TO LOOK UP
- The NHL schedule for "today" (specific date unknown, but it would correspond to the current day).
  - Source: The official NHL website (nhl.com) or an authoritative sports schedule source such as ESPN, Sportsnet, or TSN.
- Information about the teams playing, including their current statistics, rosters, standings

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)
/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_ext/agents/web_surfer/_multimodal_web_surfer.py:610: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(


---------- MultiModalMessage (MultimodalWebSurfer) ----------
I typed 'today's NHL schedule' into the browser search bar.

The web browser is open to the page [today's NHL schedule - Search](https://www.bing.com/search?q=today%27s+NHL+schedule&FORM=QBLH).
The viewport shows 24% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

By using this site you agree to the use of cookies for analytics, personalized content, and ads.
Learn more about third party cookies
|
Microsoft Privacy Policy 
Skip to content
today's NHL schedule
FrançaisMobileAll
Search
Shopping
Images
Videos
Maps
Copilot
More
Tools
About 4,480,000 resultsNHL
2024-25 Season
GAMES
NEWS
STANDINGS
Today
Game 5Canadiens
Capitals
7:00 PM
Today
Game 5Panthers
Lightning
7:30 PM
Today
Game 5Blues
Jets
9:30 PM
All times are in EST
Feedback
See more games
TSN
https://www.tsn.ca › nhl › schedule
NHL Scores and Schedules | TSN CanadaGet the latest schedule for the Natiional Hockey Le

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:304: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(self._get_compatible_context(context), json_output=True)


---------- MultiModalMessage (MultimodalWebSurfer) ----------
I typed 'Montreal Canadiens current standings notable players NHL' into the browser search bar.

The web browser is open to the page [Montreal Canadiens current standings notable players NHL - Search](https://www.bing.com/search?q=Montreal+Canadiens+current+standings+notable+players+NHL&FORM=QBLH).
The viewport shows 29% of the webpage, and is positioned at the top of the page
The following text is visible in the viewport:

By using this site you agree to the use of cookies for analytics, personalized content, and ads.
Learn more about third party cookies
|
Microsoft Privacy Policy 
Skip to content
Montreal Canadiens current standings notable players NHL
FrançaisMobileAll
Search
Shopping
Images
Videos
Maps
Copilot
More
Tools
Copilot Answer
NHL
https://www.nhl.com › canadiens › standingsMontréal Canadiens Hockey Standings - NHL.comThe official 
standings for the Montréal 
Canadiens.
How many points does Montreal Canadiens hav

/Users/arturoquiroga/3CLOUD-AUTOGEN/.venv/lib/python3.13/site-packages/autogen_agentchat/teams/_group_chat/_magentic_one/_magentic_one_orchestrator.py:455: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  response = await self._model_client.create(
